# Machine Learning - Assignment 1

## Naive Bayes Learning algorithm, Cross-validation, and ROC-Curves

The aim of the assignment is to implement:

* Naive Bayes learning algorithm for binary classification tasks
* Visualization to plot a ROC-curve
* A cross-validation test
* Visualization of the average ROC-curve of a cross-validation test

Follow the instructions and implement what is missing to complete the assignment. Some functions have been started to help you a little bit with the inputs or outputs of the function.

**Note:** You might need to go back and forth during your implementation of the code. The structure is set up to make implementation easier, but how you return values from the different functions might vary, and you might find yourself going back and change something to make it easier later on.

## Assignment preparations

We help you out with importing the libraries and reading the data.

Look at the output to get an idea of how the data is structured.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import e, pi, sqrt

data = pd.read_csv("iris.csv").to_numpy()

mapped, index, unique_arr = np.unique(data[:, -1], return_index=True, return_inverse=True)
data[:, -1] = unique_arr
iris_setosa, iris_versicolor, iris_virginica = np.split(data, index[1:])

print(f"Full data array (features and labels):\n{iris_setosa[:3]}\n")
print("###############\n")
print(f"Train features (first 4 columns):\n{iris_setosa[:3, :-1]}\n")
print(f"Labels (last column):\n{iris_setosa[:3, -1:]}\n")
print(f"Names of labels:\n{[[numb, name] for numb, name in enumerate(mapped)]}")

Full data array (features and labels):
[[5.1 3.5 1.4 0.2 0]
 [4.9 3.0 1.4 0.2 0]
 [4.7 3.2 1.3 0.2 0]]

###############

Train features (first 4 columns):
[[5.1 3.5 1.4 0.2]
 [4.9 3.0 1.4 0.2]
 [4.7 3.2 1.3 0.2]]

Labels (last column):
[[0]
 [0]
 [0]]

Names of labels:
[[0, 'Iris-setosa'], [1, 'Iris-versicolor'], [2, 'Iris-virginica']]


In [2]:
# Example print of the 3 first datapoints (similar as above):
iris_setosa[:3]

array([[5.1, 3.5, 1.4, 0.2, 0],
       [4.9, 3.0, 1.4, 0.2, 0],
       [4.7, 3.2, 1.3, 0.2, 0]], dtype=object)

## Data handling functions

As a start, we are going to implement some basic data handling functions to use in the future.

### 1) Split class into a train and test set

First, we need to be able to split the class into a train and test set.

In [21]:
def train_test_split(class_data):
    # TODO: Split the data into two parts, train and test.
    # Numpy has some functionallity that makes implementing this part easier.

    np.random.shuffle(class_data)

    train, test = np.split(class_data, 2)

    return train, test

In [22]:
# TODO: Test the train_test_split function
train, test = train_test_split(data)

# TODO: Print the output
print("train:\n", train[:5], "\ntest:\n", test[:5])

train:
 [[6.3 2.5 4.9 1.5 1]
 [6.7 3.0 5.2 2.3 2]
 [5.1 2.5 3.0 1.1 1]
 [5.6 2.7 4.2 1.3 1]
 [5.0 3.0 1.6 0.2 0]] 
test:
 [[6.2 2.9 4.3 1.3 1]
 [6.1 2.6 5.6 1.4 2]
 [6.4 3.2 5.3 2.3 2]
 [6.9 3.1 4.9 1.5 1]
 [6.3 3.4 5.6 2.4 2]]


### 2) Split data into features and labels

The data as shown above is not always the optimal shape. To help us keep track of things, we can split the data into its features and labels seperately.

Each class is 4 features and 1 label in the same array: 

* **[feature 1, feature 2, feature 3, feature 4, label]**

It would help us later to have the features and labels in seperate arrays in the form: 

* **[feature 1, feature 2, feature 3, feature 4]** and **[label]**

Here you are going to implement this functionallity.

In [28]:
def class_split(data):
    # TODO: Split one class into its features and labels

    features, labels = np.split(data, [4], axis=1)
    labels = labels.flatten()
    
    return features, labels

We should first test the "**class_split**" function on one of the classes above (iris_setosa, etc...) to make sure it works properly.

In [29]:
# TODO: Test the class splitting function
features, labels = class_split(data)

# TODO: Print the output
print("features:\n", features[:5], "\nlabels:\n", labels[:5])

features:
 [[6.3 2.5 4.9 1.5]
 [6.7 3.0 5.2 2.3]
 [5.1 2.5 3.0 1.1]
 [5.6 2.7 4.2 1.3]
 [5.0 3.0 1.6 0.2]] 
labels:
 [1 2 1 1 0]


We should also try to **1)** first split a class into a train and test set, **2)** split each of these two into features and abels. In total there should be 4 arrays (2 feature and 2 label arrays).

In [30]:
# TODO: Use the above data handling functions to make 2 feature arrays and 2 label arrays.

train, test = train_test_split(data)
train_features, train_label = class_split(train)
test_features, test_label = class_split(test)


# TODO: Print the output (you may use multiple print-statements if you wish)
print("train_features:\n", train_features[:5], "\ntrain_labels:\n", train_label[:5])
print("test_features:\n", test_features[:5], "\ntest_labels:\n", test_label[:5])


train_features:
 [[5.7 2.9 4.2 1.3]
 [6.7 2.5 5.8 1.8]
 [5.0 2.3 3.3 1.0]
 [7.4 2.8 6.1 1.9]
 [4.7 3.2 1.6 0.2]] 
train_labels:
 [1 2 1 2 0]
test_features:
 [[6.8 3.2 5.9 2.3]
 [5.5 4.2 1.4 0.2]
 [6.2 2.2 4.5 1.5]
 [4.9 3.1 1.5 0.1]
 [4.8 3.1 1.6 0.2]] 
test_labels:
 [2 0 1 0 0]


Think a bit before going to the next task, what can easily go wrong in the above code?

## Naive Bayes learning algorithm

When implementing the Navie Bayes learning algorithm, we can break it down into a few components.

We will implement these components one at a time.

### 3) Calculate feature statistics

First, we need to implement a function that returns feature statistics (means, standard deviation, priors) for a given set of feature data for a single class. This is the equivalent of "training" the naive bayes model.

**Note 1:** Each feature gets its own mean and standard deviation!

**Note 2:** The way you structure the functions (what is returned) shapes the remainder of the assignment.

In [1]:
def calculate_feature_statistics(feature_data):

    # TODO: Complete this function so that it returns the feature statistics for the input data.

    return ... # You choose how you return the statistics

To make sure the function works, we should test it before proceding.

In [2]:
# TODO: Make sure to use our previous class splitting function.

# TODO: Test the function here for one of the dataset classes. 
... = calculate_feature_statistics()


# TODO: Print the output from the feature statistic function.
print(...) 

SyntaxError: cannot assign to ellipsis here. Maybe you meant '==' instead of '='? (2942123329.py, line 4)

### 4) Gaussian probability density function (Gaussian PDF)

Now we need to implement the gaussian probability density function to use for a single datapoint.

**Note:** Look at the imports in the first cell at the top, it has some math numbers for easy use here. 

In [ ]:
def gaussian_probability_density_function(x, means, stdevs):

    prob_density = ... # TODO: Implement the gaussian probability density function.

    return prob_density

### 5) Testing Gaussian PDF

We should test it to make sure it works. Train it, using the "calculate_feature_statistics" function, on one of the dataset classes. Then, take one datapoint from the same class and use naive bayes gaussian to make a prediction.

In [ ]:
# TODO: Implement the code below to test the "gaussian_probability_density_function" function for one of the classes.

... = calculate_feature_statistics(...)

# TODO: Test with one datapoint from the learned class.
... = gaussian_probability_density_function(...)

# TODO: Print the probability density
print(...)

As a test, take one datapoint from one of the other classes and see if the predicted probability changes.

In [ ]:
# TODO: Take one datapoint from one of the other classes.
other_class_datapoint = ...

# TODO: Use naive bayes gaussian on this datapoint with the same feature statistics as the first class.
... = gaussian_probability_density_function(...)

# TODO: Print the probability
print(...)

Think a bit why the probability changes, what could affect the prediction?

## Prepare Naive Bayes for binary classification

### 6) Prepare the data for inference

Before we train and test the naive bayes for multiple classes, we should get our data in order.

Similar to how we did previously, we should now split two classes into a train and test set, you may choose which two classes freely.

In [8]:
# TODO: Split two classes into train and test sets.


# TODO: Sepearte the features and lables for both the train and test set.


### 7) Class A vs Class B for binary classification

**Note:** You might need to go back and forth a bit in the following cells during your implementation of your code.

We have to get the probability from two sets of classes and compare the two probabilities in order to make a propper prediction.

Here we will implement two functions to make this possible. We seperate these functions to make the implementation of the ROC-curve easier later on.

**Function 1: naive_bayes_prediction** 
* A function that returns the probabilities for each class the model for a single datapoint.

**Function 2: probabilities_to_prediction**
* A function that takes in probabilities and returns a prediction.

In [ ]:
def naive_bayes_prediction(feature_stats, data_point):

    # TODO: Implement the rest of this function. Make use of previous functions that you have implemented.


    # Note: We need to compare the probabilities at some point between the classes we test on, the predicted class should be the class with the highest probability.
    return prediction_probabilities

In [ ]:
def probabilities_to_prediction(probabilities):

    # TODO: Implement prediction of the class with the highest probability

    return class_prediction

To test the function we need the feature metrics from the classes we choose. 

**Note:** Choose the correct train/test set and the correct feature/label split!

In [ ]:
# TODO: Get the feature metrics for the classes.

Now we should have implemented all the neccessary parts to train a naive bayes algorithm and do inference on it. Implement a small test workflow for two of your chosen classes.

In [ ]:
# TODO: Test the "naive_bayes_prediction" function and implement all neccessary code for it to work.

# TODO: Print the predicted class and the actual class for the test set

## ROC-curve

A ROC curve, or *Receiver Operating Characteristic curve*, is a graphical plot that illustrates the performance of a binary classifier such as our Naive Bayes model.

More info can be found in the course material and here: [https://en.wikipedia.org/wiki/Receiver_operating_characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)

Another good illustration by Google can be found here: [https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

Now that we have a prediction model, we would want to try it out and test it using a ROC-curve.

### 8) True Positive Rate (TPR) and False Positive Rate (FPR)

From our prediction function we get probabilities, and for prediction purposes we have just predicted the one with the highest probability.

To plot a ROC-curve, we need the TPR and FPR for the binary classification. We will implement this here.

**Note 1:** The threshold is is a value that goes from 0 to 1. 

**Note 2:** One of the two classes will be seen as "the positive class" (prediction over the threshold) and the other as "the negative class" (prediction under the threshold).

**Note 3:** The threshold stepsize will decide the size of the returned TPR/FPR list. A value of 0.1 will give 10 elements (0 to 1 in increments of 0.1)

In [ ]:
# Stepsize demonstration
print("Python list:", [x/10 for x in range(0,10,1)])

# Stepsize demonstration with numpy:
print("Numpy linspace:", np.linspace(0,1,11))
print("Numpy linspace (no endpoint):", np.linspace(0,1,10,endpoint=False))

In [ ]:
def TPR_and_FPR(prediction_probabilities, test_labels, threshold_stepsize = 0.1):

    # TODO: Implement the rest of this function

    TPR = ...
    FPR = ...

    return TPR, FPR

In [ ]:
# TODO: Test the "TPR_and_FPR" function on the model you have created previously.

How does the values change if you change the threshold stepsize? 

How does the values change if you change the classes you compare?

### 9) Plot the TPR and FPR

To better see what is going on, we can plot the TPR and FPR. We can also calculate the Area Under the ROC Curve (AUC or AUROC) at the same time.

In [ ]:
def plot_ROC(TPR, FPR):

    # TODO: Calculate the AUC score.

    # TODO: Plot the TPR and FPR using plt (matplotlib)

    # TODO: Add a "middle-line" in the plot. This can be seen as the "better/worse than random" line.

    plt.plot(...)

    plt.show()

    pass # No need to return anything, remove this line once you are finished implementing.

In [12]:
# TODO: Test the plotting function on the TPR and FPR you just calculated.

## Cross-validation

The final task is to take everything you have implemented so far and apply it in a cross-validation loop.

**Note 1:** To better reflect a real scenarios, you should shuffle the data before doing cross-validation.

**Note 2:** When using cross-validation, the interesting thing is the mean performance (mean AUC, mean accuracy, mean ROC-curve).

**Note 3:** This part is a bit more free in terms of implementation, but make sure to use some of the previously implemented functions.

### 10) Cross-validation loop

In [ ]:
def cross_validation(data, target, folds = 10, threshold_stepsize = 0.1):

    # TODO: Implement Cross-validation
    
    for i in range(folds):


        ...

    # TODO: Plot the average ROC-curve.
    # NOTE: Take the correct average!

    plot_ROC(...)

### 11) 10-fold Cross-validation on all classes

Test the "cross_validation" function on all the classes against eachother using 10 folds.

* Iris-setosa vs Iris-versicolor
* Iris-setosa vs Iris-virginica
* Iris-versicolor vs Iris-virginica

In [13]:
# TODO: Implement and test cross-validation function on all classes.

In [ ]:
# TODO: Implement and test cross-validation function on all classes.

In [ ]:
# TODO: Implement and test cross-validation function on all classes.

# Questions for examination:

In addition to completing the assignment with all its tasks, you should also prepare to answer the following questions:

1) Why is it called "naive bayes"?

2) What are some downsides of the naive bayes learning algorithm?

3) When using ROC-curves, what is the theoretical best and worst result you can get?

4) When using ROC-curves, in this assignment for example, is a higher threshold-stepsize always better?  

5) When using cross-validation and ROC-curves, why is it important to take the correct mean values? What could go wrong?


# Finished!

Was part of the setup incorrect? Did you spot any inconsistencies in the assignment? Could something improve?

If so, please write them and send via email and send it to:

* marcus.gullstrand@ju.se

Thank you!